In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os

def create_folders(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully!")
    else:
        print(f"Folder '{folder_name}' already exists!")

results_folder = 'Results_with_single_model'
create_folders(results_folder)

pseudo_df = pd.read_csv("pseudodataE288_BQM.csv")

# Load the single precomputed model
single_model = tf.keras.models.load_model("single_model_with_mean_std.h5")
print("Single model loaded successfully!")

# Define analytical function for fDNNQ(QM)
def fDNNQ(QM, b=0.5):
    """Analytical function for fDNNQ(QM)"""
    return np.exp(-b * QM)

# Generate QM values for comparison
QM_values = np.linspace(pseudo_df['QM'].min(), pseudo_df['QM'].max(), 200)
QM_tensor = QM_values.reshape(-1, 1)

# Compute mean and std for DNNQ predictions
dnnQ_contributions = np.array([model.predict(QM_tensor, verbose=0).flatten() for model in models_list])
dnnQ_mean = np.mean(dnnQ_contributions, axis=0)
dnnQ_std = np.std(dnnQ_contributions, axis=0)
fDNNQ_values = fDNNQ(QM_values)

# Plot comparison of analytical and DNNQ model predictions
plt.figure(figsize=(10, 6))
plt.plot(QM_values, fDNNQ_values, label=r'Analytical $\mathcal{B}(Q_M)$', linestyle='--', color='blue')
plt.plot(QM_values, dnnQ_mean, label='DNNQ Model Mean', linestyle='-', color='red')
plt.fill_between(QM_values, dnnQ_mean - dnnQ_std, dnnQ_mean + dnnQ_std, color='red', alpha=0.2, label="DNNQ Std Dev")
plt.xlabel(r'$Q_M$', fontsize=14)
plt.ylabel(r'$f_{DNNQ}(Q_M)$', fontsize=14)
plt.title('Comparison of Analytical $\mathcal{B}(Q_M)$ and DNNQ Model', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.savefig(f"{results_folder}/QM_comparison_plot.pdf")
plt.show()


# Function to compute theoretical A values using single model
def compute_A(x1, x2, qT, QM):
    """ Compute theoretical A values using the pre-trained single model """
    f_u_x1 = pdf(NNPDF4_nlo, 2, x1, QM)
    f_ubar_x2 = pdf(NNPDF4_nlo, -2, x2, QM)
    f_u_x2 = pdf(NNPDF4_nlo, 2, x2, QM)
    f_ubar_x1 = pdf(NNPDF4_nlo, -2, x1, QM)

    Sk_contribution = (1 / 2) * np.pi * np.exp(-qT ** 2 / 2)

    QM_input = QM.reshape(-1, 1)
    fDNN_mean, fDNN_std = single_model.predict(QM_input, verbose=0)

    ux1ubarx2_term = x1 * x2 * f_u_x1 * f_ubar_x2 * Sk_contribution
    ubarx1ux2_term = x2 * x1 * f_u_x2 * f_ubar_x1 * Sk_contribution

    FUU_mean = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_mean.flatten()
    cross_section_mean = FUU_mean * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM ** 3)

    FUU_std = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_std.flatten()
    cross_section_std = FUU_std * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM ** 3)

    return cross_section_mean, cross_section_std


# Compute A_pred and A_std for the entire dataset
pseudo_df["A_pred"], pseudo_df["A_std"] = compute_A(
    pseudo_df['x1'].values, pseudo_df['x2'].values, pseudo_df['qT'].values, pseudo_df['QM'].values
)

# Save results
pseudo_df.to_csv(f"{results_folder}/Results_pseudodataE288_with_LHAPDF.csv", index=False)
print("Results saved successfully!")


# Prepare data grouping for plotting
pseudo_df["unique_group"] = (
    pseudo_df["QM"].astype(str) + "_" + pseudo_df["x1"].astype(str) + "_" + pseudo_df["x2"].astype(str)
)
groups = pseudo_df.groupby("unique_group")
n_groups = groups.ngroups
ncols = 3
nrows = (n_groups + ncols - 1) // ncols

# Plot results
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5 * nrows))
axes = axes.flatten()

for idx, (group_name, group_df) in enumerate(groups):
    qT = group_df['qT'].values
    QM = group_df['QM'].values
    x1 = group_df['x1'].values
    x2 = group_df['x2'].values
    A_true = group_df['A'].values

    A_pred, A_std = compute_A(x1, x2, qT, QM)

    axes[idx].errorbar(qT, A_pred, yerr=A_std, fmt='o', color='red', label='A_pred with error bars')
    axes[idx].plot(qT, A_true, '.', color='blue', label='A_true')
    axes[idx].set_title(f'A vs qT for $Q_M$ = {QM[0]:.2f} GeV')
    axes[idx].set_xlabel('qT')
    axes[idx].set_ylabel('A')
    axes[idx].legend()
    axes[idx].grid(True)

plt.tight_layout()
plt.savefig(f"{results_folder}/QM_subplots_with_predictions_with_LHAPDF.pdf")
plt.show()


Folder 'Results_with_single_model' already exists!
Single model loaded successfully!


NameError: name 'models_list' is not defined

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os

def create_folders(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully!")
    else:
        print(f"Folder '{folder_name}' already exists!")

results_folder = 'Results_with_single_model'
create_folders(results_folder)

pseudo_df = pd.read_csv("pseudodataE288_BQM.csv")

# Load the single precomputed model
single_model = tf.keras.models.load_model("single_model_with_mean_std.h5")
print("Single model loaded successfully!")

# Define models_list with the single model
models_list = [single_model]  # Fixing the undefined 'models_list' error

# Define analytical function for fDNNQ(QM)
def fDNNQ(QM, b=0.5):
    return np.exp(-b * QM)

# Generate QM values for comparison
QM_values = np.linspace(pseudo_df['QM'].min(), pseudo_df['QM'].max(), 200)
QM_tensor = QM_values.reshape(-1, 1)

# Compute mean and std for DNNQ predictions
dnnQ_contributions = np.array([model.predict(QM_tensor, verbose=0).flatten() for model in models_list])
dnnQ_mean = np.mean(dnnQ_contributions, axis=0)
dnnQ_std = np.std(dnnQ_contributions, axis=0)
fDNNQ_values = fDNNQ(QM_values)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(QM_values, fDNNQ_values, label=r'Analytical $\mathcal{B}(Q_M)$', linestyle='--', color='blue')
plt.plot(QM_values, dnnQ_mean, label='DNNQ Model Mean', linestyle='-', color='red')
plt.fill_between(QM_values, dnnQ_mean - dnnQ_std, dnnQ_mean + dnnQ_std, color='red', alpha=0.2, label="DNNQ Std Dev")
plt.xlabel(r'$Q_M$', fontsize=14)
plt.ylabel(r'$f_{DNNQ}(Q_M)$', fontsize=14)
plt.title('Comparison of Analytical $\mathcal{B}(Q_M)$ and DNNQ Model', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.savefig(f"{results_folder}/QM_comparison_plot.pdf")
plt.show()




# Function to compute theoretical A values using single model
def compute_A(x1, x2, qT, QM):
    """ Compute theoretical A values using the pre-trained single model """
    f_u_x1 = pdf(NNPDF4_nlo, 2, x1, QM)
    f_ubar_x2 = pdf(NNPDF4_nlo, -2, x2, QM)
    f_u_x2 = pdf(NNPDF4_nlo, 2, x2, QM)
    f_ubar_x1 = pdf(NNPDF4_nlo, -2, x1, QM)

    Sk_contribution = (1 / 2) * np.pi * np.exp(-qT ** 2 / 2)

    QM_input = QM.reshape(-1, 1)
    #fDNN_mean, fDNN_std = single_model.predict(QM_input, verbose=0)
    fDNN_mean, fDNN_std = single_model.predict(QM_input, verbose=0)
    fDNN_mean = np.array(fDNN_mean).flatten()
    fDNN_std = np.array(fDNN_std).flatten()

    ux1ubarx2_term = x1 * x2 * f_u_x1 * f_ubar_x2 * Sk_contribution
    ubarx1ux2_term = x2 * x1 * f_u_x2 * f_ubar_x1 * Sk_contribution

    FUU_mean = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_mean.flatten()
    cross_section_mean = FUU_mean * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM ** 3)

    FUU_std = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_std.flatten()
    cross_section_std = FUU_std * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM ** 3)

    return cross_section_mean, cross_section_std


# Compute A_pred and A_std for the entire dataset
pseudo_df["A_pred"], pseudo_df["A_std"] = compute_A(
    pseudo_df['x1'].values, pseudo_df['x2'].values, pseudo_df['qT'].values, pseudo_df['QM'].values
)

# Save results
pseudo_df.to_csv(f"{results_folder}/Results_pseudodataE288_with_LHAPDF.csv", index=False)
print("Results saved successfully!")


# Prepare data grouping for plotting
pseudo_df["unique_group"] = (
    pseudo_df["QM"].astype(str) + "_" + pseudo_df["x1"].astype(str) + "_" + pseudo_df["x2"].astype(str)
)
groups = pseudo_df.groupby("unique_group")
n_groups = groups.ngroups
ncols = 3
nrows = (n_groups + ncols - 1) // ncols

# Plot results
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5 * nrows))
axes = axes.flatten()

for idx, (group_name, group_df) in enumerate(groups):
    qT = group_df['qT'].values
    QM = group_df['QM'].values
    x1 = group_df['x1'].values
    x2 = group_df['x2'].values
    A_true = group_df['A'].values

    A_pred, A_std = compute_A(x1, x2, qT, QM)

    axes[idx].errorbar(qT, A_pred, yerr=A_std, fmt='o', color='red', label='A_pred with error bars')
    axes[idx].plot(qT, A_true, '.', color='blue', label='A_true')
    axes[idx].set_title(f'A vs qT for $Q_M$ = {QM[0]:.2f} GeV')
    axes[idx].set_xlabel('qT')
    axes[idx].set_ylabel('A')
    axes[idx].legend()
    axes[idx].grid(True)

plt.tight_layout()
plt.savefig(f"{results_folder}/QM_subplots_with_predictions_with_LHAPDF.pdf")
plt.show()



Folder 'Results_with_single_model' already exists!
Single model loaded successfully!


AttributeError: 'list' object has no attribute 'flatten'